In [ ]:

# 导入必要的库
import os
import scipy.stats
import numpy as np
import pandas as pd


In [ ]:

# 文件路径设置
BASEDIR = ''  # 设置适当的目录路径
DATADIR = os.path.normpath(os.path.join(BASEDIR, 'data'))
ALL_GENES_FNAME = os.path.join(DATADIR, 'DrugBank_TTD_target_genelist.txt')
ATC_TARGETS_FNAME = os.path.join(DATADIR, 'DrugBank_TTD_targets_by_ATC_v2.txt')
ATC_ANNOT_FNAME = os.path.join(DATADIR, 'ATC_annotation.txt')
ICD_TARGETS_FNAME = os.path.join(DATADIR, 'DrugBank_TTD_targets_by_ICD10.txt')
ICD_ANNOT_FNAME = os.path.join(DATADIR, 'icd10_annotation_category.txt')

# 手动指定参数（替代 argparse 的用法）
genelist = 'path_to_genelist.txt'  # 基因列表文件路径
out_prefix = 'output_prefix'  # 输出文件前缀
background = None  # 可选的背景文件
test_type = 'ATC'  # 选择测试类型：'ATC' 或 'ICD'
output_drug_name = False  # 是否包含药物名称信息


In [ ]:

# 定义 grep 函数
def grep(target, key, annot, out_drug=False):
    # 提取目标数据
    group = target[key].iloc[0]
    this_group_genes = target.TargetGene.unique()
    joined_genes = np.intersect1d(target_gene, this_group_genes)
    classified_genes = np.intersect1d(all_genes, this_group_genes)

    # 计算列举基因的交集
    a = len(joined_genes)
    b = len(classified_genes) - a
    c = len(target_analysis_genes) - a
    d = len(all_genes) - len(target_analysis_genes) - b

    # 计算费舍尔精确检验的结果
    oddsratio, pvalue = scipy.stats.fisher_exact([[a, b], [c, d]], alternative="greater")

    # 输出结果列
    out_cols = ['#Group', 'GroupName', 'OddsRatio', 'FisherExactP']
    if out_drug:
        out_cols.append('TargetGene:DrugNames')
        gene_druglist = target[target.TargetGene.isin(joined_genes)].groupby('TargetGene').apply(
            lambda x: x.TargetGene.iloc[0] + ':' + ','.join(x.Drug.unique()))
        drugnames = ";".join(gene_druglist)
        ret = [group, annot.Annot[group], oddsratio, pvalue, drugnames]
    else:
        ret = [group, annot.Annot[group], oddsratio, pvalue]
    return pd.Series(ret, out_cols)


In [ ]:

# 处理 ATC 和 ICD 测试逻辑

# 加载基因列表和定义变量
target_gene = pd.read_csv(genelist, header=None)[0].unique()
all_genes = pd.read_table(ALL_GENES_FNAME, header=None)[0]

# 如果有背景文件，则进行交集操作
if background is not None:
    all_defined = pd.read_csv(background, header=None)[0].unique()
    all_genes = np.intersect1d(all_genes, all_defined)
target_analysis_genes = np.intersect1d(all_genes, target_gene)

# 处理 ATC 测试
if test_type == "ATC":
    ATC = pd.read_csv(ATC_TARGETS_FNAME, header=None, sep='	',
                      names=['Code', 'Drug', 'TargetGene', 'XXX']).dropna(subset=['TargetGene'])
    ATC['large'] = ATC.Code.str.slice(0, 1)
    ATC_ANNOT = pd.read_csv(ATC_ANNOT_FNAME, header=None, sep='	', index_col=0, names=['Code', 'Annot'])

    # 对大组进行分析
    ATC.groupby('large').apply(
        grep, annot=ATC_ANNOT, key='large',
        out_drug=output_drug_name).to_csv(
            out_prefix + ".ATC.large.txt", index=False, sep='	')

    # 对详细组进行分析
    ATC.groupby('Code').apply(
        grep, annot=ATC_ANNOT, key='Code',
        out_drug=output_drug_name).to_csv(
            out_prefix + ".ATC.detail.txt", index=False, sep='	')

# 处理 ICD 测试
elif test_type == "ICD":
    annot_icd = {}
    subcat_icd = {}
    with open(ICD_ANNOT_FNAME, "r") as f:
        for line in f:
            line = line.strip().split("	")
            annot_icd[line[0]] = line[1]
            [subcat_icd.update({v: line[0]}) for v in line[2:]]
    ICD_ANNOT = pd.DataFrame.from_dict(
        annot_icd, orient='index').rename(columns={0: 'Annot'})

    ICD = pd.read_csv(ICD_TARGETS_FNAME, header=None, sep='	', names=['Code', 'Drug', 'TargetGene'])
    ICD['large'] = ICD.Code.apply(
        lambda x: subcat_icd.setdefault(x, np.nan))
    ICD = ICD.dropna()

    ICD.groupby('large').apply(
        grep, annot=ICD_ANNOT, key='large',
        out_drug=output_drug_name).to_csv(
            out_prefix + ".ICD.txt", index=False, sep='	')
